In [15]:
import pickle
import os
from sklearn.metrics import roc_auc_score


# Path to the directory containing the pickle files
pickle_path = '/ceph/hpc/data/st2207-pgp-users/ldragar/Marijaproject/SelfBlendedImages/preds'

# List all files in the directory
pickle_files = os.listdir(pickle_path)

# Dictionary to hold combined predictions
all_preds = []

for file in pickle_files:
    file_path = os.path.join(pickle_path, file)
    with open(file_path, 'rb') as f:
        preds = pickle.load(f)
        all_preds.append(preds)
        
        

#concat dataframes
import pandas as pd

df = pd.concat(all_preds)

print(df.head())

print(df.shape)

df["vid_basename"] = df["video"].apply(lambda x: os.path.basename(x))

print(df.head())

#load josns
import json

train = json.load(open('/ceph/hpc/data/st2207-pgp-users/ldragar/Marijaproject/train.json'))
val = json.load(open('/ceph/hpc/data/st2207-pgp-users/ldragar/Marijaproject/val.json'))
test = json.load(open('/ceph/hpc/data/st2207-pgp-users/ldragar/Marijaproject/test.json'))

train_files = ["_".join(x) for x in train]
val_files = ["_".join(x) for x in val]
test_files = ["_".join(x) for x in test]

#add reverse pairs
train_files += ["_".join(x[::-1]) for x in train]
val_files += ["_".join(x[::-1]) for x in val]
test_files += ["_".join(x[::-1]) for x in test]

print(train_files[:5])
print(val_files[:5])
print(test_files[:5])

#add mp4 extension
train_files = [x+'.mp4' for x in train_files]
val_files = [x+'.mp4' for x in val_files]
test_files = [x+'.mp4' for x in test_files]

#set split in df map to vid_basename

df['split'] = 'train'
df.loc[df['vid_basename'].isin(val_files), 'split'] = 'val'
df.loc[df['vid_basename'].isin(test_files), 'split'] = 'test'

print(df.head())











                                               video  \
0  /ceph/hpc/data/st2207-pgp-users/ldragar/Marija...   
1  /ceph/hpc/data/st2207-pgp-users/ldragar/Marija...   
2  /ceph/hpc/data/st2207-pgp-users/ldragar/Marija...   
3  /ceph/hpc/data/st2207-pgp-users/ldragar/Marija...   
4  /ceph/hpc/data/st2207-pgp-users/ldragar/Marija...   

                                               preds      mean  
0  [0.0015004805, 0.0016265878, 0.001493684, 0.00...  0.009082  
1  [0.031192096, 0.033630483, 0.033017725, 0.0338...  0.087194  
2  [0.033167, 0.022576077, 0.014114569, 0.0157985...  0.219651  
3  [0.030185832, 0.063520424, 0.09086239, 0.06501...  0.341747  
4  [0.012509033, 0.02412499, 0.04122225, 0.072030...  0.070354  
(889, 3)
                                               video  \
0  /ceph/hpc/data/st2207-pgp-users/ldragar/Marija...   
1  /ceph/hpc/data/st2207-pgp-users/ldragar/Marija...   
2  /ceph/hpc/data/st2207-pgp-users/ldragar/Marija...   
3  /ceph/hpc/data/st2207-pgp-users/ldrag

In [16]:
df

,video,preds,mean,vid_basename,split
0,/ceph/hpc/data/st2207-pgp-users/ldragar/Marija...,"[0.0015004805, 0.0016265878, 0.001493684, 0.00...",0.009082,875_979.mp4,val
1,/ceph/hpc/data/st2207-pgp-users/ldragar/Marija...,"[0.031192096, 0.033630483, 0.033017725, 0.0338...",0.087194,876_891.mp4,train
2,/ceph/hpc/data/st2207-pgp-users/ldragar/Marija...,"[0.033167, 0.022576077, 0.014114569, 0.0157985...",0.219651,877_886.mp4,train
3,/ceph/hpc/data/st2207-pgp-users/ldragar/Marija...,"[0.030185832, 0.063520424, 0.09086239, 0.06501...",0.341747,878_866.mp4,train
4,/ceph/hpc/data/st2207-pgp-users/ldragar/Marija...,"[0.012509033, 0.02412499, 0.04122225, 0.072030...",0.070354,879_963.mp4,train
...,...,...,...,...,...
120,/ceph/hpc/data/st2207-pgp-users/ldragar/Marija...,"[0.05894179, 0.3045615, 0.142717, 0.17484589, ...",0.173461,495_512.mp4,train
121,/ceph/hpc/data/st2207-pgp-users/ldragar/Marija...,"[0.031682562, 0.008170093, 0.022714317, 0.0242...",0.062016,496_417.mp4,train
122,/ceph/hpc/data/st2207-pgp-users/ldragar/Marija...,"[0.18497309, 0.6068921, 0.11871336, 0.51166993...",0.394452,497_403.mp4,train
123,/ceph/hpc/data/st2207-pgp-users/ldragar/Marija...,"[0.0038879209, 0.004541858, 0.020956991, 0.021...",0.197337,498_433.mp4,train


In [13]:
#get counts
print(df['split'].value_counts())

#calc auc on val

val_scores = df[df['split']=='val']['mean'].values
train_scores = df[df['split']=='train']['mean'].values
test_scores = df[df['split']=='test']['mean'].values
print(val_scores.shape)


split
train    627
val      122
test     115
Name: count, dtype: int64
(122,)


In [14]:
#plotly plot
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace( go.Histogram(x=val_scores, histnorm='probability', name='val'))
fig.add_trace( go.Histogram(x=train_scores, histnorm='probability', name='train'))
fig.add_trace( go.Histogram(x=test_scores, histnorm='probability', name='test'))

fig.update_layout(title='Distribution mean scores', xaxis_title='Mean score', yaxis_title='Density')

fig.show()





